In [18]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from numpy import linalg as LA
from abc import abstractmethod

# # suppress warnings
# import warnings
# warnings.filterwarnings("ignore")

plt.style.use('dark_background')

In [19]:
class MatrixSolver:

    # compute gradinent of function f in point x with step h
    @staticmethod
    def grad(x, f, eps):
        derivative = np.arange(np.size(x))
        for i in range(np.size(x)):
            x[i] += eps
            f1 = f(x)
            x[i] -= 2 * eps
            f2 = f(x)
            x[i] += eps
            derivative[i] = (f1 - f2) / (2 * eps)
        return derivative

    # compute Jacobian of functions \sigma{r1...rn} in point x
    @staticmethod
    def findJacobian(rs, x, eps):
        J = np.asarray([])
        for r in rs:
            J = np.append(J, MatrixSolver.grad(x, r, eps))
        return J

    # compute pseudo inverse of matrix X
    @staticmethod
    def pseudoInverse(x):
        return np.linalg.inv(x.T @ x) @ x.T

    # compute hessian of functions \sigma{r1...rn} in point x
    @staticmethod
    def findHessian(rs, x):
        J = MatrixSolver.findJacobian(rs, x)
        return 2 * (J.T @ J)

    # get function in vector form
    @staticmethod
    def function_array(x):
       return np.array([1 - x[0] , x[1] - x[0] ** 2]).reshape((2,1))


In [20]:
class Searcher:
    def __init__(self, eps, max_iterations):
        self.eps = eps
        self.max_iterations = max_iterations

    # stop method
    def is_not_final(self):
        return LA.norm(self.next - self.cur) > self.eps

    # executor method
    def run(self, start_x, func, lr_func, start_lr):
        self.epoch = 0
        self.lr = start_lr
        self.func = func
        self.cur_x = start_x
        self.lr_func = lr_func
    
        self.next_x = self.next_point()
        while (self.is_not_final()):
            self.epoch = self.epoch + 1
            self.cur_x = next
        return self.next


    # find next point
    @abstractmethod
    def next_point(self):
        pass

    


In [21]:
class GaussNewton(Searcher):
  
    def next_point(self):
        self.lr = self.lr_func(self.lr, self.epoch)
        p = MatrixSolver.pseudoInverse(MatrixSolver.findJacobian(self.func, self.cur_x, self.eps))
        self.next_x = self.cur_x - self.lr * np.dot(p, self.function_array(self.x)).reshape((-1))

